In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from googletrans import Translator
translator = Translator()
data = pd.read_csv("/content/251DM_Research (1).csv")
translated_data = pd.DataFrame(columns=["review"])
i=0
total = data.shape[0]
for index, row in data.iterrows():
    review = translator.translate(row["review"], dest="en").text
    if translator.detect(review).lang == "en" :
        translated_data.at[i, "review"] = review
        i = i+1

with open("translations.csv", "w") as f:
    translated_data.to_csv(f)

#translated_data.head(10)

data["review_en"] = translated_data.review
data.head(10)

,review,sentiment,review_en
0,এই টাকায় ভালো পন্য আপনার নিতে পারেন,Positive,This money can take good products
1,আলহামদুলিল্লাহ অনেক ভালো একটা প্রোডাক্ট। সবাই ...,Positive,Alhamdulillah is a very good product.Everyone ...
2,সেলারকে অনেক অনেক ধন্যবাদ ভালো মান সম্মত পণ্য ...,Positive,Thanks to Salar very much for good quality pro...
3,আপনাদের পন্য আমি পেয়েছি। তবে একটা বিষয় হলো পন্...,Positive,I got your products.But one thing that we woul...
4,26 তারিখ অর্ডার দিয়েছে 29 তারিখ পেয়েছি খুব ভ...,Positive,The 26th order received 29 dates got a very go...
5,অনেক আহামরি টাইপের ভালো না। কিন্তু দাম অনুযায়ী...,Positive,Many Ahmori types are not good.But according t...
6,সেলার কে কথা দিয়েছিলাম যদি ভালো প্রডাক্ট পাই ত...,Positive,"If you had a good product in Salar, then you m..."
7,কালারগুলো ঠিকঠাক আসেনি। তারপরও মোটামুটি ভাল ছি...,Positive,Colors did not come properly.Still was fairly ...
8,দাম অনুযায়ী খারাপ না। তবে এই ধরণের প্রোডাক্ট অ...,Positive,"The price is not bad.However, everyone is inte..."
9,প্রোডাক্ট কোয়ালিটি অনেক ভালো। তারথেকেও বেশী ভ...,Positive,Product quality is very good.Messenger Respons...


In [ ]:
#Using nltk to load the sentiment analyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#loading sentiment Analyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
#lower and upper thresholds
threshold_lower = 0.4
threshold_upper = 0.85

In [ ]:
scores = []
rating = []

#iterating every review
for i in data["review_en"]:
    #calculating the sentiment score and comparing threshold
    if sid.polarity_scores(i)["compound"] < threshold_lower:
        rating.append("-1")
    elif sid.polarity_scores(i)["compound"] < threshold_upper:
        rating.append("1")
    else:
        rating.append("1")
    #appending scores
    scores.append(sid.polarity_scores(i)["compound"])

#loading rating score and rating to dataframe
data["rating_score"] = scores
data["rating"] = rating
data.head(10)

,review,sentiment,review_en,rating_score,rating
0,এই টাকায় ভালো পন্য আপনার নিতে পারেন,Positive,This money can take good products,0.4404,1
1,আলহামদুলিল্লাহ অনেক ভালো একটা প্রোডাক্ট। সবাই ...,Positive,Alhamdulillah is a very good product.Everyone ...,0.4927,1
2,সেলারকে অনেক অনেক ধন্যবাদ ভালো মান সম্মত পণ্য ...,Positive,Thanks to Salar very much for good quality pro...,0.8639,1
3,আপনাদের পন্য আমি পেয়েছি। তবে একটা বিষয় হলো পন্...,Positive,I got your products.But one thing that we woul...,0.4576,1
4,26 তারিখ অর্ডার দিয়েছে 29 তারিখ পেয়েছি খুব ভ...,Positive,The 26th order received 29 dates got a very go...,0.8221,1
5,অনেক আহামরি টাইপের ভালো না। কিন্তু দাম অনুযায়ী...,Positive,Many Ahmori types are not good.But according t...,0.6784,1
6,সেলার কে কথা দিয়েছিলাম যদি ভালো প্রডাক্ট পাই ত...,Positive,"If you had a good product in Salar, then you m...",0.9366,1
7,কালারগুলো ঠিকঠাক আসেনি। তারপরও মোটামুটি ভাল ছি...,Positive,Colors did not come properly.Still was fairly ...,0.0000,-1
8,দাম অনুযায়ী খারাপ না। তবে এই ধরণের প্রোডাক্ট অ...,Positive,"The price is not bad.However, everyone is inte...",0.6597,1
9,প্রোডাক্ট কোয়ালিটি অনেক ভালো। তারথেকেও বেশী ভ...,Positive,Product quality is very good.Messenger Respons...,-0.2810,-1


In [ ]:
data.head(5)

,review,sentiment,review_en,rating_score,rating
0,এই টাকায় ভালো পন্য আপনার নিতে পারেন,Positive,This money can take good products,0.4404,1
1,আলহামদুলিল্লাহ অনেক ভালো একটা প্রোডাক্ট। সবাই ...,Positive,Alhamdulillah is a very good product.Everyone ...,0.4927,1
2,সেলারকে অনেক অনেক ধন্যবাদ ভালো মান সম্মত পণ্য ...,Positive,Thanks to Salar very much for good quality pro...,0.8639,1
3,আপনাদের পন্য আমি পেয়েছি। তবে একটা বিষয় হলো পন্...,Positive,I got your products.But one thing that we woul...,0.4576,1
4,26 তারিখ অর্ডার দিয়েছে 29 তারিখ পেয়েছি খুব ভ...,Positive,The 26th order received 29 dates got a very go...,0.8221,1


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data[['rating_score']], data['sentiment'], random_state=0)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [ ]:
y_pred = knn.predict(x_test)

In [ ]:
pd.concat([x_test, y_test, pd.Series(y_pred, name='Predicted', index=x_test.index)], 
          ignore_index=False, axis=1)

,rating_score,sentiment,Predicted
225,-0.3412,Negative,Negative
122,0.5574,Positive,Positive
92,0.4927,Positive,Positive
157,0.0000,Positive,Positive
154,0.0000,Positive,Positive
...,...,...,...
188,0.7003,Positive,Positive
15,0.8591,Positive,Positive
217,0.4754,Positive,Positive
74,0.8173,Positive,Positive


In [ ]:
print("Test set score: {:.2f}".format(knn.score(x_test, y_test)))

Test set score: 0.83


In [ ]:
from sklearn import svm
 
clf = svm.SVC(kernel='linear') 
 
clf.fit(x_train, y_train)
 
 
y_pred = clf.predict(x_test)

In [ ]:
from sklearn import metrics
 
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7936507936507936


In [ ]:
from sklearn.ensemble import RandomForestClassifier
 
clf=RandomForestClassifier(n_estimators=100)
 
clf.fit(x_train,y_train)

y_pred=clf.predict(x_test)

In [ ]:
from sklearn import metrics 
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7619047619047619


In [ ]:
from sklearn.naive_bayes import GaussianNB
 
gnb = GaussianNB()
 
gnb.fit(x_train, y_train)
 
y_pred = gnb.predict(x_test)

In [ ]:
from sklearn import metrics
 
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7936507936507936
